In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
from numpy.fft import fft, ifft
from scipy.signal import find_peaks
from scipy import signal
from scipy.signal import argrelextrema
py.init_notebook_mode(connected=True)
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
#from catboost import CatBoostClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

In [4]:
df = pd.read_csv('/content/drive/MyDrive/PCOMP/testes/testesComRuidos/datasetSemRuido.csv')
df

,f_60,f_120,f_180,f_240,f_300,f_360,f_420,f_480,f_540,f_600,...,f_720,f_780,f_840,f_900,f_960,f_1020,f_1080,f_1140,f_1200,device
0,0.131348,0.013978,0.012229,0.008640,0.025304,0.006685,0.006257,0.005137,0.000000,0.004323,...,0.002947,0.003268,0.000000,0.002524,0.003185,0.003144,0.000000,0.002709,0.003098,airConditioner
1,0.131721,0.000000,0.019724,0.000000,0.012496,0.000000,0.000000,0.005493,0.005682,0.004079,...,0.003569,0.004057,0.003879,0.003709,0.003862,0.000000,0.003381,0.003252,0.003166,airConditioner
2,0.183198,0.015159,0.008908,0.000000,0.015539,0.007133,0.007818,0.004496,0.003655,0.003613,...,0.002022,0.002000,0.001797,0.001677,0.002288,0.001935,0.001454,0.001722,0.001816,airConditioner
3,0.141297,0.000000,0.013481,0.000000,0.014397,0.007276,0.000000,0.005555,0.005213,0.004371,...,0.003455,0.003841,0.003234,0.003176,0.000000,0.000000,0.003351,0.002887,0.003081,airConditioner
4,0.126771,0.017283,0.015065,0.012812,0.029172,0.007726,0.009169,0.006785,0.005399,0.000000,...,0.004564,0.000000,0.003991,0.004232,0.000000,0.003511,0.003031,0.000000,0.003661,airConditioner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.001105,0.000000,0.001310,0.000310,0.001316,0.000276,0.000696,0.000244,0.000224,0.000141,...,0.000350,0.000000,0.000000,0.000422,0.000000,0.000372,0.000000,0.000174,0.000246,monitor
1596,0.001213,0.000000,0.001419,0.000000,0.001130,0.000177,0.000581,0.000207,0.000289,0.000194,...,0.000058,0.000233,0.000128,0.000380,0.000000,0.000260,0.000189,0.000207,0.000000,monitor
1597,0.001405,0.000446,0.001329,0.000194,0.001245,0.000258,0.000681,0.000155,0.000361,0.000345,...,0.000000,0.000347,0.000283,0.000476,0.000000,0.000285,0.000347,0.000283,0.000349,monitor
1598,0.001273,0.000000,0.001329,0.000000,0.001119,0.000000,0.000680,0.000000,0.000351,0.000165,...,0.000000,0.000200,0.000202,0.000306,0.000000,0.000230,0.000000,0.000309,0.000000,monitor


In [5]:
classes = ["microwave", "airConditioner", "lamp", "monitor"]
X = df.drop(columns=['device'])
y = df['device'].apply(classes.index).to_numpy()
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

## Random Forest

In [6]:
GRID = [
    {
        'scaler': [StandardScaler()],
        'estimator': [RandomForestClassifier(random_state=42)],
        'estimator__n_estimators': [50, 100, 200],        # Número de árvores na floresta
        'estimator__max_depth': [None, 10, 20, 30],      # Profundidade máxima das árvores
        'estimator__min_samples_split': [2, 5, 10],      # Número mínimo de amostras necessárias para dividir um nó
        'estimator__min_samples_leaf': [1, 2, 4]         # Número mínimo de amostras necessárias para ser uma folha
    }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', RandomForestClassifier())])

In [7]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Melhor F1-SCORE:  0.9937676943117673


In [8]:
best_model.best_params_

{'estimator': RandomForestClassifier(n_estimators=200, random_state=42),
 'estimator__max_depth': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__n_estimators': 200,
 'scaler': StandardScaler()}

In [9]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.996874984741062
Acurácia:  0.996875


## Gradient Boosting

In [10]:
GRID = [
    {
        'scaler': [StandardScaler()],
        'estimator': [GradientBoostingClassifier(random_state=42)],
        'estimator__n_estimators': [50, 100, 200],        # Número de estimadores (árvores)
        'estimator__learning_rate': [0.01, 0.1, 0.2],    # Taxa de aprendizado
        'estimator__max_depth': [3, 4, 5],               # Profundidade máxima das árvores de decisão
        'estimator__min_samples_split': [2, 3, 4]        # Número mínimo de amostras necessárias para dividir um nó
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', GradientBoostingClassifier())])

In [11]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Melhor F1-SCORE:  0.9978926353149955


In [12]:
best_model.best_params_

{'estimator': GradientBoostingClassifier(learning_rate=0.2, max_depth=4, random_state=42),
 'estimator__learning_rate': 0.2,
 'estimator__max_depth': 4,
 'estimator__min_samples_split': 2,
 'estimator__n_estimators': 100,
 'scaler': StandardScaler()}

In [13]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.9937596136996437
Acurácia:  0.99375


## Decision Tree

In [14]:
GRID = [
    {
        'scaler': [StandardScaler()],
        'estimator': [DecisionTreeClassifier(random_state=42)],
        'estimator__criterion': ['gini', 'entropy'],    # Critério de divisão
        'estimator__max_depth': [None, 10, 20, 30],     # Profundidade máxima da árvore
        'estimator__min_samples_split': [2, 5, 10],    # Número mínimo de amostras para dividir um nó
        'estimator__min_samples_leaf': [1, 2, 4]       # Número mínimo de amostras em um nó folha

     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', DecisionTreeClassifier())])

In [15]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Melhor F1-SCORE:  0.9947866865525118


In [16]:
best_model.best_params_

{'estimator': DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2,
                        min_samples_split=5, random_state=42),
 'estimator__criterion': 'entropy',
 'estimator__max_depth': None,
 'estimator__min_samples_leaf': 2,
 'estimator__min_samples_split': 5,
 'scaler': StandardScaler()}

In [17]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.9875288082795226
Acurácia:  0.9875
